In [1]:
!pip install transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

The output from model is wrapped in a markdown tag ```python ```, use regex to parse it

In [3]:
import re

def extract_python_code(text: str) -> str:
    """Extracts Python code enclosed in triple backticks from a given text."""
    match = re.search(r"```python\s*(.*?)\s*```", text, re.DOTALL)
    return match.group(1) if match else ""

Prompt to convert other language to Python:
``` text
You are a helpful assistant that excellent in converting codes.

I will give you a code snippet, if it is not a Python code, convert it to executable Python code.

Do not include explanations, restate the question.

Do not add quotation marks or punctuation—just the letter. Just the Python code.
```

In [4]:
def get_response(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that excellent in converting codes. I will give you a code snippet, if it is not a Python code, convert it to executable Python code. Do not include explanations, restate the question. Do not add quotation marks or punctuation—just the letter. Just the Python code."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return extract_python_code(response)

A small function to extract question that need code executing from test_set

In [ ]:
import csv
import re

def extract_questions(input_csv, output_csv):
    patterns = [
        r"^Question: What will be output\??",
        r"^Question: What will be the output\??",
        r"^Question: What is the output\??",
        r"^Question: What would be the output\??"
    ]

    with open(input_csv, newline='', encoding='utf-8') as infile, \
         open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:

        reader = csv.DictReader(infile)
        fieldnames = ['task_id', 'extracted_text']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            question = row['question'].strip()
            task_id = row['task_id']

            for pattern in patterns:
                match = re.match(pattern, question)
                if match:
                    extracted_text = question[question.find('?')+1:].strip()
                    writer.writerow({'task_id': task_id, 'extracted_text': extracted_text})
                    break

extract_questions('b6_test_data.csv', 'b6_code_extracted.csv')

Interate through question and convert them to Python executable code.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("/content/b6_code_extracted.csv")

results = []
for _, row in tqdm(df.iterrows()):
    task_id = row['task_id']
    question = row['extracted_text']

    result = get_response(question)
    print(result)

    # save task_id and answer to results to save to submission.csv file
    results.append({
        'task_id': task_id,
        'python_code': result
    })
    # Save the results to a CSV file
output_df = pd.DataFrame(results)
    # save to csv
output_df.to_csv('python_code_converted.csv', index=False)

1it [00:28, 28.10s/it]

import sys

a = sys.getsizeof(int) + sys.getsizeof(int)
b = sys.getsizeof(int) + sys.getsizeof(int)
print(a + b)


2it [01:06, 34.20s/it]

SIZE = 5
expr = 1 | 2 | 3 | 4
value = [2.0, 4.0, 6.0, 8.0, 10.0]
print(value[expr])


3it [01:28, 28.54s/it]

...

class Good:
    def __init__(self, m):
        self.m = m

    def equals(self, n):
        return n.m == self.m

if __name__ == "__main__":
    m1 = Good(22)
    m2 = Good(22)
    s1 = Good(22)
    s2 = Good(22)

    print(m1.equals(m2))
    print(m1.equals(s2))
    print(s1.equals(m2))
    print(s1.equals(s2))


195it [2:06:02, 35.72s/it]

total = 0
for num in range(1, 6):
    total += num
print(total)


196it [2:06:54, 38.85s/it]

import React from 'react'
import ReactDOM from 'react-dom'

class HelloWorld extends React.Component {
    render() {
        return (
            <div>
                Hello World 1
                <div>Hello World 2</div>
            </div>
        )
    }
}

const mountNode = document.getElementById('mountNode')
ReactDOM.render(<HelloWorld />, mountNode)


Execute the python code, then the output with the choices are fed into the model for final results.

In [ ]:
import os
import pandas as pd
import signal
import traceback
from tqdm import tqdm

def safe_exec(code):
    """Safely execute Python code with a timeout."""
    def handler(signum, frame):
        raise TimeoutError("Execution timed out")

    signal.signal(signal.SIGALRM, handler)
    signal.alarm(5)  # Set a 5-second timeout

    local_vars = {}
    try:
        exec(code, {}, local_vars)
        signal.alarm(0)  # Disable alarm after successful execution
        return local_vars
    except Exception as e:
        return str(e)

# Load converted Python codes
df_codes = pd.read_csv("python_code_converted.csv")

# Load test data containing questions and choices
df_test = pd.read_csv("b6_test_data.csv")

# Dictionary to quickly look up questions and choices
test_data_map = {
    row['task_id']: (row['question'], eval(row['choices']))  # Ensure choices are properly parsed
    for _, row in df_test.iterrows()
}

results = []

for _, row in tqdm(df_codes.iterrows()):
    task_id = row['task_id']
    python_code = row['python_code']

    if task_id not in test_data_map:
        continue

    question, choices = test_data_map[task_id]

    # Execute the Python code safely
    exec_result = safe_exec(python_code)

    # Format the prompt for the model
    prompt = (
        "I will give you the question that needs executing a code, some choices, "
        "and the code running result. Tell me which answer in the choices is correct "
        "under an uppercase letter showing the position of the choice in choices.\n\n"
        f"Question: {question}\n"
        f"Choices: {choices}\n"
        f"Execution Result: {exec_result}"
    )

    # Get model response
    answer = get_response(prompt)

    # Store the results
    results.append({
        'task_id': task_id,
        'answer': answer
    })

# Save results to CSV
output_df = pd.DataFrame(results)
output_df.to_csv('submission.csv', index=False)